# Filtering and Sorting Data

This time we are going to pull data directly from the internet.

### Step 1. Import the necessary libraries

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder.appName("Euro").getOrCreate()

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/02_Filtering_%26_Sorting/Euro12/Euro_2012_stats_TEAM.csv).

### Step 3. Assign it to a variable called euro12.

In [8]:
!wget https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/02_Filtering_%26_Sorting/Euro12/Euro_2012_stats_TEAM.csv -O euro

--2025-06-20 09:09:27--  https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/02_Filtering_%26_Sorting/Euro12/Euro_2012_stats_TEAM.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2319 (2.3K) [text/plain]
Saving to: ‘euro’

euro                100%[===================>]   2.26K  --.-KB/s    in 0s      

2025-06-20 09:09:27 (29.8 MB/s) - ‘euro’ saved [2319/2319]



In [9]:
euro12 = spark.read.format("csv") \
.option("header","true") \
.option("inferSchema","true") \
.load("euro")

### Step 4. Select only the Goal column.

In [11]:
euro12.select("Goals").show()

+-----+
|Goals|
+-----+
|    4|
|    4|
|    4|
|    5|
|    3|
|   10|
|    5|
|    6|
|    2|
|    2|
|    6|
|    1|
|    5|
|   12|
|    5|
|    2|
+-----+



In [12]:
euro12.columns

['Team',
 'Goals',
 'Shots on target',
 'Shots off target',
 'Shooting Accuracy',
 '% Goals-to-shots',
 'Total shots (inc. Blocked)',
 'Hit Woodwork',
 'Penalty goals',
 'Penalties not scored',
 'Headed goals',
 'Passes',
 'Passes completed',
 'Passing Accuracy',
 'Touches',
 'Crosses',
 'Dribbles',
 'Corners Taken',
 'Tackles',
 'Clearances',
 'Interceptions',
 'Clearances off line',
 'Clean Sheets',
 'Blocks',
 'Goals conceded',
 'Saves made',
 'Saves-to-shots ratio',
 'Fouls Won',
 'Fouls Conceded',
 'Offsides',
 'Yellow Cards',
 'Red Cards',
 'Subs on',
 'Subs off',
 'Players Used']

### Step 5. How many team participated in the Euro2012?

In [14]:
euro12.select("Team").distinct().count()

16

### Step 6. What is the number of columns in the dataset?

In [15]:
print(len(euro12.columns))

35


### Step 7. View only the columns Team, Yellow Cards and Red Cards and assign them to a dataframe called discipline

In [16]:
discipline = euro12.select("Team","Yellow Cards","Red Cards")
discipline.show()

+-------------------+------------+---------+
|               Team|Yellow Cards|Red Cards|
+-------------------+------------+---------+
|            Croatia|           9|        0|
|     Czech Republic|           7|        0|
|            Denmark|           4|        0|
|            England|           5|        0|
|             France|           6|        0|
|            Germany|           4|        0|
|             Greece|           9|        1|
|              Italy|          16|        0|
|        Netherlands|           5|        0|
|             Poland|           7|        1|
|           Portugal|          12|        0|
|Republic of Ireland|           6|        1|
|             Russia|           6|        0|
|              Spain|          11|        0|
|             Sweden|           7|        0|
|            Ukraine|           5|        0|
+-------------------+------------+---------+



### Step 8. Sort the teams by Red Cards, then to Yellow Cards

In [26]:
discipline.orderBy(col("Red Cards").desc(), col("Yellow Cards").desc()).show()
# Here we have passed two cols in order as needed, to sort first by red cards and then by yellow cards

+-------------------+------------+---------+
|               Team|Yellow Cards|Red Cards|
+-------------------+------------+---------+
|             Greece|           9|        1|
|             Poland|           7|        1|
|Republic of Ireland|           6|        1|
|              Italy|          16|        0|
|           Portugal|          12|        0|
|              Spain|          11|        0|
|            Croatia|           9|        0|
|     Czech Republic|           7|        0|
|             Sweden|           7|        0|
|             France|           6|        0|
|             Russia|           6|        0|
|            England|           5|        0|
|        Netherlands|           5|        0|
|            Ukraine|           5|        0|
|            Denmark|           4|        0|
|            Germany|           4|        0|
+-------------------+------------+---------+



### Step 9. Calculate the mean Yellow Cards given per Team

In [31]:
discipline.groupBy("Team").agg(mean("Yellow Cards").alias("Mean_Yel_cards")).show()
# OR discipline.agg(mean("Yellow Cards").alias("Mean_yellow_cards")).show()

+-------------------+--------------+
|               Team|Mean_Yel_cards|
+-------------------+--------------+
|             Russia|           6.0|
|             Sweden|           7.0|
|            Germany|           4.0|
|             France|           6.0|
|             Greece|           9.0|
|            Croatia|           9.0|
|              Italy|          16.0|
|              Spain|          11.0|
|            Denmark|           4.0|
|            Ukraine|           5.0|
|     Czech Republic|           7.0|
|Republic of Ireland|           6.0|
|            England|           5.0|
|             Poland|           7.0|
|           Portugal|          12.0|
|        Netherlands|           5.0|
+-------------------+--------------+



### Step 10. Filter teams that scored more than 6 goals

In [32]:
euro12.where(col("Goals") > 6).show()

+-------+-----+---------------+----------------+-----------------+----------------+--------------------------+------------+-------------+--------------------+------------+------+----------------+----------------+-------+-------+--------+-------------+-------+----------+-------------+-------------------+------------+------+--------------+----------+--------------------+---------+--------------+--------+------------+---------+-------+--------+------------+
|   Team|Goals|Shots on target|Shots off target|Shooting Accuracy|% Goals-to-shots|Total shots (inc. Blocked)|Hit Woodwork|Penalty goals|Penalties not scored|Headed goals|Passes|Passes completed|Passing Accuracy|Touches|Crosses|Dribbles|Corners Taken|Tackles|Clearances|Interceptions|Clearances off line|Clean Sheets|Blocks|Goals conceded|Saves made|Saves-to-shots ratio|Fouls Won|Fouls Conceded|Offsides|Yellow Cards|Red Cards|Subs on|Subs off|Players Used|
+-------+-----+---------------+----------------+-----------------+----------------

### Step 11. Select the teams that start with G

In [35]:
euro12.where(col("Team").like("G%")).show()

+-------+-----+---------------+----------------+-----------------+----------------+--------------------------+------------+-------------+--------------------+------------+------+----------------+----------------+-------+-------+--------+-------------+-------+----------+-------------+-------------------+------------+------+--------------+----------+--------------------+---------+--------------+--------+------------+---------+-------+--------+------------+
|   Team|Goals|Shots on target|Shots off target|Shooting Accuracy|% Goals-to-shots|Total shots (inc. Blocked)|Hit Woodwork|Penalty goals|Penalties not scored|Headed goals|Passes|Passes completed|Passing Accuracy|Touches|Crosses|Dribbles|Corners Taken|Tackles|Clearances|Interceptions|Clearances off line|Clean Sheets|Blocks|Goals conceded|Saves made|Saves-to-shots ratio|Fouls Won|Fouls Conceded|Offsides|Yellow Cards|Red Cards|Subs on|Subs off|Players Used|
+-------+-----+---------------+----------------+-----------------+----------------

### Step 12. Select the first 7 columns

In [39]:
euro12.columns[:7]

['Team',
 'Goals',
 'Shots on target',
 'Shots off target',
 'Shooting Accuracy',
 '% Goals-to-shots',
 'Total shots (inc. Blocked)']

### Step 13. Select all columns except the last 3.

In [42]:
euro12.columns[:-3]

['Team',
 'Goals',
 'Shots on target',
 'Shots off target',
 'Shooting Accuracy',
 '% Goals-to-shots',
 'Total shots (inc. Blocked)',
 'Hit Woodwork',
 'Penalty goals',
 'Penalties not scored',
 'Headed goals',
 'Passes',
 'Passes completed',
 'Passing Accuracy',
 'Touches',
 'Crosses',
 'Dribbles',
 'Corners Taken',
 'Tackles',
 'Clearances',
 'Interceptions',
 'Clearances off line',
 'Clean Sheets',
 'Blocks',
 'Goals conceded',
 'Saves made',
 'Saves-to-shots ratio',
 'Fouls Won',
 'Fouls Conceded',
 'Offsides',
 'Yellow Cards',
 'Red Cards']

### Step 14. Present only the Shooting Accuracy from England, Italy and Russia

In [47]:
euro12.select("Team","Shooting Accuracy").where(col("Team").isin("England","Italy","Russia")).show()

+-------+-----------------+
|   Team|Shooting Accuracy|
+-------+-----------------+
|England|            50.0%|
|  Italy|            43.0%|
| Russia|            22.5%|
+-------+-----------------+

